In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
import json
import random

from numpy import array
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
import joblib
import re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model
import tensorflow
from tensorflow.python.keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def trainIntentModel():
    dataset = pd.read_csv('/content/drive/MyDrive/Agric_Chatbot/intent.csv', names=["Query", "Intent"])
    X = dataset["Query"]
    y = dataset["Intent"]

    queryCorpus = []
    ps = PorterStemmer()

    for query in X:
        query = re.sub('[^a-zA-Z]', ' ', query)
        query = ' '.join([ps.stem(word.lower()) for word in query.split()])
        queryCorpus.append(query)

    countVectorizer = CountVectorizer(max_features=800)
    corpus = countVectorizer.fit_transform(queryCorpus).toarray()

    joblib.dump(countVectorizer, "/content/drive/MyDrive/Agric_Chatbot/IntentCountVectorizer.sav")

    labelencoder_intent = LabelEncoder()
    y = to_categorical(labelencoder_intent.fit_transform(y))

    intent_label_map = {cl: labelencoder_intent.transform([cl])[0] for cl in labelencoder_intent.classes_}

    classifier = Sequential()
    classifier.add(Dense(units=96, kernel_initializer='uniform', activation='relu', input_dim=corpus.shape[1]))
    classifier.add(Dense(units=96, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=32, kernel_initializer='uniform', activation='softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    classifier.fit(corpus, y, batch_size=10, epochs=500)

    return classifier, intent_label_map

In [ ]:
def trainIntentModel():
    dataset = pd.read_csv('/content/drive/MyDrive/Agric_Chatbot/intent.csv', names=["Query", "Intent"])
    X = dataset["Query"]
    y = dataset["Intent"]

    queryCorpus = []
    ps = PorterStemmer()

    for query in X:
        query = ' '.join([ps.stem(word.lower()) for word in re.sub('[^a-zA-Z]', ' ', query).split()])
        queryCorpus.append(query)

    countVectorizer = CountVectorizer(max_features=800)
    corpus = countVectorizer.fit_transform(queryCorpus).toarray()

    joblib.dump(countVectorizer, "/content/drive/MyDrive/Agric_Chatbot/IntentCountVectorizer.sav")

    labelencoder_intent = LabelEncoder()
    y = to_categorical(labelencoder_intent.fit_transform(y))

    intent_label_map = {cl: labelencoder_intent.transform([cl])[0] for cl in labelencoder_intent.classes_}

    classifier = Sequential()
    classifier.add(Dense(units=96, kernel_initializer='uniform', activation='relu', input_dim=corpus.shape[1]))
    classifier.add(Dense(units=96, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dense(units=32, kernel_initializer='uniform', activation='softmax'))
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    classifier.fit(corpus, y, batch_size=10, epochs=500)

    return classifier, intent_label_map

def trainEntityModel():
    dataset = pd.read_csv('/content/drive/MyDrive/Agric_Chatbot/agric-data.csv', names=["word", "label"])
    X = dataset["word"]
    y = dataset["label"]

    entityCorpus = []
    ps = PorterStemmer()

    for word in X.astype(str):
        word = ps.stem(word)
        entityCorpus.append(word)

    cv = CountVectorizer(max_features=1500)
    X = cv.fit_transform(entityCorpus).toarray()

    joblib.dump(cv, '/content/drive/MyDrive/Agric_Chatbot/CountVectorizer.sav')

    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y.astype(str))

    entity_label_map = {cl: labelencoder_y.transform([cl])[0] for cl in labelencoder_y.classes_}

    classifier = GaussianNB()
    classifier.fit(X, y)

    joblib.dump(classifier, '/content/drive/MyDrive/Agric_Chatbot/entity_model.sav')

    return entity_label_map

intent_model, intent_label_map = trainIntentModel()
intent_model.save('/content/drive/MyDrive/Agric_Chatbot/intent_model.h5')
entity_label_map = trainEntityModel()

Epoch 1/500
21/21 [==============================] - 2s 4ms/step - loss: 3.4605 - accuracy: 0.0766
Epoch 2/500
21/21 [==============================] - 0s 4ms/step - loss: 3.4078 - accuracy: 0.1100
Epoch 3/500
21/21 [==============================] - 0s 3ms/step - loss: 3.2100 - accuracy: 0.0813
Epoch 4/500
21/21 [==============================] - 0s 3ms/step - loss: 2.9285 - accuracy: 0.1292
Epoch 5/500
21/21 [==============================] - 0s 3ms/step - loss: 2.6753 - accuracy: 0.2249
Epoch 6/500
21/21 [==============================] - 0s 4ms/step - loss: 2.4393 - accuracy: 0.3014
Epoch 7/500
21/21 [==============================] - 0s 4ms/step - loss: 2.2150 - accuracy: 0.4067
Epoch 8/500
21/21 [==============================] - 0s 4ms/step - loss: 1.9836 - accuracy: 0.4689
Epoch 9/500
21/21 [==============================] - 0s 4ms/step - loss: 1.7169 - accuracy: 0.5359
Epoch 10/500
21/21 [==============================] - 0s 4ms/step - loss: 1.4501 - accuracy: 0.6986
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
entity_label_map = trainEntityModel()
loadedEntityCV = pk.load(open('/content/drive/MyDrive/Agric_Chatbot/EntityCountVectorizer.sav', 'rb'))
loadedEntityClassifier = joblib.load(open('/content/drive/MyDrive/Agric_Chatbot/entity_model.sav', 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.23.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
def getEntities(query):
    query = loadedEntityCV.transform(query).toarray()
    response_tags = loadedEntityClassifier.predict(query)
    entity_list = [list(entity_label_map.keys())[list(entity_label_map.values()).index(tag)] for tag in response_tags if tag in entity_label_map.values()]
    return entity_list

In [ ]:
with open('/content/drive/MyDrive/Agric_Chatbot/intents.json') as json_data:
    intents = json.load(json_data)

loadedIntentClassifier = load_model('/content/drive/MyDrive/Agric_Chatbot/intent_model.h5')
loaded_intent_CV = joblib.load('/content/drive/MyDrive/Agric_Chatbot/IntentCountVectorizer.sav')

USER_INTENT = ""

while True:
    user_query = input()

    query = re.sub('[^a-zA-Z]', ' ', user_query).split(' ')
    ps = PorterStemmer()
    tokenized_query = [ps.stem(word.lower()) for word in query]

    processed_text = loaded_intent_CV.transform([' '.join(tokenized_query)]).toarray()
    predicted_Intent = loadedIntentClassifier.predict(processed_text)
    result = np.argmax(predicted_Intent, axis=1)

    for key, value in intent_label_map.items():
        if value == result[0]:
            USER_INTENT = key
            break

    for i in intents['intents']:
        if i['tag'] == USER_INTENT:
            print(random.choice(i['responses']))

    entities = getEntities(tokenized_query)
    token_entity_map = dict(zip(entities, tokenized_query))

1/1 [==============================] - 0s 58ms/step
A temperature of 21°C- 27°C and 50-100cm rainfall is suitable for its cultivation.
1/1 [==============================] - 0s 20ms/step
minimum and maximum growth temperatures of 3° to 4°C and 30° to 32°C, respectively.
1/1 [==============================] - 0s 28ms/step
The average temperature required throughout the life period of the crop ranges from 21 to 37º C.


https://colab.research.google.com/drive/1dH_dXxhEmmfVr6cTUJ95ggXLQSQPDh-c?usp=sharing